In [25]:
import pandas as pd
from ast import literal_eval
import numpy as np
from scipy.spatial.distance import euclidean, pdist, squareform
from os.path import join as os_join
import seaborn as sns
import matplotlib.pylab as plt
from pyvis import network as net


pd.set_option('display.max_colwidth', None)

In [26]:
similarity_path = "../data/processed/vis_dataset/similarity_doc2vec.csv"
dataset_path = "../data/processed/vis_dataset/vis_data.csv"

output_path = "../reports/figures/doc2vec_similarity/similarity_Doc2Vec.html"

In [27]:
dataset = pd.read_csv(dataset_path)

dataset["Title"] = dataset["Title"].astype(str)
dataset["AuthorNames"] = dataset["AuthorNames"].astype(str)


dataset.sample(2)

,Conference,Year,Title,DOI,id,cite_to_list,cited_by_list,Link,FirstPage,LastPage,...,Abstract,AuthorNames-Deduped,AuthorNames,AuthorAffiliation,InternalReferences,AuthorKeywords,AminerCitationCount_02-2020,XploreCitationCount - 2020-01,PubsCited,Award
807,VAST,2012,The User Puzzle---Explaining the Interaction with Visual Analytics Systems,10.1109/TVCG.2012.273,807,"[419, 681, 624, 307, 383]","[872, 1040, 1204, 1438, 1486]",http://dx.doi.org/10.1109/TVCG.2012.273,2908,2916,...,"Visual analytics emphasizes the interplay between visualization, analytical procedures performed by computers and human perceptual and cognitive activities. Human reasoning is an important element in this context. There are several theories in psychology and HCI explaining open-ended and exploratory reasoning. Five of these theories (sensemaking theories, gestalt theories, distributed cognition, graph comprehension theories and skill-rule-knowledge models) are described in this paper. We discuss their relevance for visual analytics. In order to do this more systematically, we developed a schema of categories relevant for visual analytics research and evaluation. All these theories have strengths but also weaknesses in explaining interaction with visual analytics systems. A possibility to overcome the weaknesses would be to combine two or more of these theories.",Margit Pohl;Michael Smuc;Eva Mayr,Margit Pohl;Michael Smuc;Eva Mayr,Vienna University of Technology;Danube University Krems;Danube University Krems,10.1109/TVCG.2008.121;10.1109/TVCG.2007.70515;10.1109/VAST.2010.5653598;10.1109/VAST.2008.4677361;10.1109/VAST.2011.6102445,"Cognitive theory, visual knowledge discovery, interaction design, reasoning, problem solving",18.0,18.0,65.0,NaN
1730,InfoVis,2019,Construct-A-Vis: Exploring the Free-Form Visualization Processes of Children,10.1109/TVCG.2019.2934804,1730,[],[],http://dx.doi.org/10.1109/TVCG.2019.2934804,451,460,...,"Building data analysis skills is part of modern elementary school curricula. Recent research has explored how to facilitate children's understanding of visual data representations through completion exercises which highlight links between concrete and abstract mappings. This approach scaffolds visualization activities by presenting a target visualization to children. But how can we engage children in more free-form visual data mapping exercises that are driven by their own mapping ideas? How can we scaffold a creative exploration of visualization techniques and mapping possibilities? We present Construct-A-Vis, a tablet-based tool designed to explore the feasibility of free-form and constructive visualization activities with elementary school children. Construct-A-Vis provides adjustable levels of scaffolding visual mapping processes. It can be used by children individually or as part of collaborative activities. Findings from a study with elementary school children using Construct-A-Vis individually and in pairs highlight the potential of this free-form constructive approach, as visible in children's diverse visualization outcomes and their critical engagement with the data and mapping processes. Based on our study findings we contribute insights into the design of free-form visualization tools for children, including the role of tool-based scaffolding mechanisms and shared interactions to guide visualization activities with children.",Fearn Bishop;Johannes Zagermann;Ulrike Pfeil;Gemma Sanderson;Harald Reiterer;Uta Hinrichs,Fearn Bishop;Johannes Zagermann;Ulrike Pfeil;Gemma Sanderson;Harald Reiterer;Uta Hinrichs,"University of St Andrews, St Andrews, UK;University of Konstanz, Konstanz, Germany;University of Konstanz, Konstanz, Germany;Fife Council, UK;University of Konstanz, Konstanz, Germany;University of St Andrews, St Andrews, UK",NaN,"Visualization in Education,Visualization with Children,Qualitative Evaluation,Visualization System and Toolkit Design",0.0,0.0,NaN,NaN


In [28]:
id_title_abstract = "<b>Paper id</b>: " + dataset["id"].astype(str) + " ("  + dataset["Conference"] + ") " " - "\
"<a href=\""  + dataset.Link + "\" target=\"_blank\">Link</a>" + \
"; <br><b>Authors: </b>: " + dataset["AuthorNames"] + \
"; <br><b>Title</b>: " + dataset["Title"] + \
"; <br><b>Abstract</b>: " + dataset["Abstract"]

map_id_to_label={k: v for  k,v in enumerate(id_title_abstract)}
map_id_to_label[0]

'<b>Paper id</b>: 0 (InfoVis)  - <a href="http://dx.doi.org/10.1109/INFVIS.1995.528680" target="_blank">Link</a>; <br><b>Authors: </b>: B. Spence;L. Tweedie;H. Dawkes;Hua Su; <br><b>Title</b>: Visualisation for functional design; <br><b>Abstract</b>: We present two novel visualisation tools: the Influence Explorer and the Prosection Matrix. These were specifically created to support engineering artifact design and similar tasks in which a set of parameter values must be chosen to lead to acceptable artifact performance. These tools combine two concepts. One is the interactive and virtually immediate responsive display of data in a manner conducive to the acquisition of insight. The other, involving the precalculation of samples of artifact performance, facilitates smooth exploration and optimisation leading to a design decision. The anticipated benefits of these visualisation tools are illustrated by an example taken from electronic circuit design, in which full account must be taken o

In [29]:
df = pd.read_csv(similarity_path)
df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,1736,1737,1738,1739,1740,1741,1742,1743,1744,1745
0,0,-1.000000,0.568328,0.462271,0.454161,0.642155,0.540096,0.551160,0.468066,0.390690,...,0.355781,0.456573,0.568745,0.630027,0.402972,0.444379,0.558462,0.557408,0.560330,0.496985
1,1,0.568328,-1.000000,0.327876,0.514606,0.296113,0.381139,0.490727,0.506431,0.262349,...,0.541579,0.507317,0.338627,0.570750,0.223580,0.119695,0.601685,0.377666,0.311815,0.488700
2,2,0.462271,0.327876,-1.000000,0.528790,0.258828,0.613659,0.638728,0.544126,0.383845,...,0.288476,0.524038,0.333307,0.262148,0.579781,0.486939,0.251883,0.420083,0.374725,0.461788
3,3,0.454161,0.514606,0.528790,-1.000000,0.500015,0.569363,0.540131,0.480091,0.479014,...,0.515064,0.421636,0.511353,0.379185,0.534273,0.527216,0.442845,0.702163,0.657056,0.439307
4,4,0.642155,0.296113,0.258828,0.500015,-1.000000,0.553881,0.645605,0.158116,0.642725,...,0.341039,0.323165,0.467028,0.343945,0.418254,0.549289,0.367514,0.668831,0.550037,0.438186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1741,1741,0.444379,0.119695,0.486939,0.527216,0.549289,0.515795,0.546503,0.379432,0.466030,...,0.352275,0.191376,0.514101,0.318249,0.593109,-1.000000,0.452709,0.589822,0.691349,0.522740
1742,1742,0.558462,0.601685,0.251883,0.442845,0.367514,0.264349,0.447632,0.403387,0.271778,...,0.614482,0.455766,0.656377,0.409905,0.269491,0.452709,-1.000000,0.496920,0.534461,0.421971
1743,1743,0.557408,0.377666,0.420083,0.702163,0.668831,0.549612,0.687164,0.272261,0.595197,...,0.403439,0.452550,0.710840,0.381544,0.669028,0.589822,0.496920,-1.000000,0.797513,0.552322
1744,1744,0.560330,0.311815,0.374725,0.657056,0.550037,0.490000,0.615911,0.308767,0.421654,...,0.576391,0.356563,0.710266,0.339426,0.571327,0.691349,0.534461,0.797513,-1.000000,0.475081


In [30]:
fill_diagonal = -1
thres_to_plot = 0.87

    
arr = df.values
index_names = df.index
col_names = df.columns

#  Get indices where such threshold is crossed; avoid diagonal elems
R,C = np.where(np.triu(arr,1)>=thres_to_plot)

# Arrange those in columns and put out as a dataframe
out_arr = np.column_stack((index_names[R],col_names[C],arr[R,C]))
df_out = pd.DataFrame(out_arr,columns=['from','to','value'])
df_out["from"] = df_out["from"].astype(np.int16)
df_out["to"] = df_out["to"].astype(np.int16)
df_out

,from,to,value
0,21,754,0.878708
1,21,1453,0.891444
2,25,1048,0.879187
3,30,831,0.887319
4,36,1331,0.870539
...,...,...,...
387,1565,1629,0.88035
388,1598,1701,0.875607
389,1666,1667,0.89219
390,1678,1713,0.879727


## Plot using Pyvis

In [31]:
color_dict = {
    "InfoVis": "blue",
    "VAST": "orange",
    "SciVis": "red"
}

def get_node_color(node, df, color_dict):
    conference = df[df.id == node].Conference.values[0]
    return color_dict[conference]

In [32]:
thres = thres_to_plot
df_out_thres = df_out[df_out["value"] >= thres]
plotting_all_nodes = False

file_name= output_path
# g = net.Network(height='95%', width='99%',heading='Paper Similarity Using #BibliographicCouplings and #Co-citations')

g = net.Network(height="700px", width="100%",heading='Paper Similarity Using #BibliographicCouplings and #Co-citations')
g.set_edge_smooth('discrete')
# g.set_options('{"nodes": {"borderWidthSelected": "10" } }')

if plotting_all_nodes:
    node_list = list(df.index.astype(np.int16))
else:
    node_list = list(set.union( set(df_out_thres["from"]), set(df_out_thres["to"]) ) )

node_labels = [ map_id_to_label.get(node) for node in node_list]
node_colors = [ get_node_color(node, dataset, color_dict) for node in node_list]
    
# g.add_nodes(node_list, title=node_labels, label=node_list, color=node_colors)

g.add_nodes(node_list, title=node_labels, color=node_colors)


edge_list = zip(df_out_thres["from"], df_out_thres["to"], df_out_thres['value'])
g.add_edges(edge_list)    
# g.show_buttons(filter_=["physics", "edges"])
# g.show_buttons()


g.show(file_name)

#  source code: https://github.com/WestHealth/pyvis/blob/523d541d04df6779dd21e5942af60b79369e8f13/pyvis/network.py#L218

In [33]:
g.num_nodes()

412

In [ ]:
"structure" in 

In [47]:
import dill
#### Write
with open('/Users/chaupham/minhchau/MY_WORK/C/citation_networks/models/network_plot/doc2vec_similarity.pkl', 'wb') as f:     
    dill.dump(g, f)

In [42]:
!open "/Users/chaupham/minhchau/MY_WORK/C/citation_networks/reports/figures/doc2vec_similarity"

In [15]:
g.nodes[0].update({"color":"green"})

In [48]:
"scape Maps" in g.nodes[0].get("title")

True

In [54]:
file_name

'../reports/figures/ref_similarity/similarity_using_citeTo_and_citedBy.html'

In [50]:
!open $file_name

In [52]:
!open .

## Plot using Jaal

More elegant and faster, but still in its infancy though. There's a lack of some crucial features such as "edge thickness"

In [12]:
dataset

,Conference,Year,Title,DOI,id,cite_to_list,cited_by_list,Link,FirstPage,LastPage,...,Abstract,AuthorNames-Deduped,AuthorNames,AuthorAffiliation,InternalReferences,AuthorKeywords,AminerCitationCount_02-2020,XploreCitationCount - 2020-01,PubsCited,Award
0,InfoVis,1995,Visualisation for functional design,10.1109/INFVIS.1995.528680,0,[],[990],http://dx.doi.org/10.1109/INFVIS.1995.528680,4,10,...,"We present two novel visualisation tools: the Influence Explorer and the Prosection Matrix. These were specifically created to support engineering artifact design and similar tasks in which a set of parameter values must be chosen to lead to acceptable artifact performance. These tools combine two concepts. One is the interactive and virtually immediate responsive display of data in a manner conducive to the acquisition of insight. The other, involving the precalculation of samples of artifact performance, facilitates smooth exploration and optimisation leading to a design decision. The anticipated benefits of these visualisation tools are illustrated by an example taken from electronic circuit design, in which full account must be taken of the uncertainties in parameter values arising from inevitable variations in the manufacturing process.",Robert Spence;Lisa Tweedie;Huw Dawkes;Hua Su,B. Spence;L. Tweedie;H. Dawkes;Hua Su,"Dept. of Electr. Eng., Imperial Coll. of Sci., Technol. & Med., London, UK;Dept. of Electr. Eng., Imperial Coll. of Sci., Technol. & Med., London, UK;Dept. of Electr. Eng., Imperial Coll. of Sci., Technol. & Med., London, UK;Dept. of Electr. Eng., Imperial Coll. of Sci., Technol. & Med., London, UK",NaN,NaN,NaN,12.0,10.0,NaN
1,InfoVis,1995,Towards a generative theory of diagram design,10.1109/INFVIS.1995.528681,1,[],[],http://dx.doi.org/10.1109/INFVIS.1995.528681,11,18,...,"We describe the theoretical background for AVE, an automatic visualization engine for semantic networks. We have a functional notion of aesthetics and therefore understand meaningfulness as a central issue for information visualization. This implies that the diagrams should communicate the characteristics of the data as effectively as possible. In this generative theory of diagram design, we include data characterization, systematic use of graphical means of expression and the combination of graphical means of expression. After giving a brief introduction and an application scenario we discuss these aspects in detail. Finally, a process model of an automatic visualization process is sketched and directions for further research are outlined.",Klaus Reichenberger;Thomas Kamps;Gene Golovchinsky,K. Reichenberger;T. Kamps;G. Golovchinsky,"GMD-Inst. for Integrated Publication & Inf. Sci., Darmstadt, Germany;GMD-Inst. for Integrated Publication & Inf. Sci., Darmstadt, Germany;GMD-Inst. for Integrated Publication & Inf. Sci., Darmstadt, Germany",10.1109/VISUAL.1995.480815,NaN,12.0,4.0,18.0,NaN
2,InfoVis,1995,Research report: information animation applications in the capital markets,10.1109/INFVIS.1995.528682,2,[],"[32, 52]",http://dx.doi.org/10.1109/INFVIS.1995.528682,19,25,...,"3D computer graphics can be extremely expressive. It is possible to display an entire securities market, like the S&amp;P 500, on a single screen. With the correct approach to the visual design of the layout, these massive amounts of information can be quickly and easily comprehended by a human observer. By using motion and animated interaction, it is possible to use 3D as a reliable, accurate and precise decision-support tool. Information animation applications are particularly suited to the securities industry because that is where we find huge amounts of data, the value of which declines rapidly with time, and where critical decisions are being made on this data in very short periods of time. Information animation technology is an important new tool for the securities industry, where people need to be in the decision-making loop without suffering from information overload. Sever

In [ ]:
import pandas as pd
from jaal import Jaal
from jaal.datasets import load_got


thres = 6
plotting_all_nodes = False


    
    

df_out_thres = df_out[df_out["value"] >= thres]
edges = df_out_thres

(dataset)
# edges.loc[:, 'label'] = edges.loc[:, 'value'].astype(int).astype(str)

edges["from"] = edges["from"].map(lambda node_id: " - ".join(dataset[dataset["id"] == node_id ][["Title", "AuthorNames"]].values[0]))
edges["to"] = edges["to"].map(lambda node_id: " - ".join(dataset[dataset["id"] == node_id ][["Title", "AuthorNames"]].values[0])) 


set_thres_nodes = set.union( set(df_out_thres["from"]), set(df_out_thres["to"]) )
nodes = pd.DataFrame({"id": df.index})
nodes["having_edge"] = nodes["id"].map(lambda x: x in set_thres_nodes).astype(str)
nodes["conference"] = nodes["id"].map(lambda node_id: dataset[dataset["id"] == node_id ]["Conference"].values[0])
nodes["Title"] = nodes["id"].map(lambda node_id: " - ".join(dataset[dataset["id"] == node_id ][["Title", "AuthorNames"]].values[0])) 
nodes["id"] = nodes["Title"]
if plotting_all_nodes:
    pass
else:
    print(nodes.shape)
    node_list = set.union( set(df_out_thres["from"]), set(df_out_thres["to"]) ) 
    nodes = nodes[nodes["id"].isin(node_list)]
    print(nodes.shape)

nodes.to_csv("../data/interim/jaal_data/nodes_2.csv", index=False)
edges.to_csv("../data/interim/jaal_data/edges_2.csv", index=False)

port=8050

while True:
    try:
        Jaal(edges, nodes).plot(port=port)
    except:
        port+=1


/Users/chaupham/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/chaupham/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(1746, 4)
(294, 4)
Parsing the data...Done
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/chaupham/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/chaupham/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Apr/2021 00:26:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:26:53] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:26:53] "GET /_dash-layout HT

No trigger


127.0.0.1 - - [13/Apr/2021 00:26:54] "GET /_favicon.ico?v=1.19.0 HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:26:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:27:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:27:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:27:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:27:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:27:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:27:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:27:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:27:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:27:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:27:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.

In [ ]:
edges["value"] = edges["value"].astype(int)

In [ ]:
edges.nlargest(10,"value")

In [ ]:
node_list

In [ ]:
nodes[nodes.id==1606].Title.values[0]

In [ ]:
edges[(edges["to"] == 755) & (edges["from"]==456)]